# Trabajo Práctico 1 
### Clasificación sobre datos simulados. 

## Introducción
Para este trabajo, hemos creado una función generadora de minions. Sobre cada minion, hemos medido 200 características que representan habilidades que poseen en distintas tareas (relacionadas al Mal).  

El doctor Nefario ha ideado una fórmula para determinar si un minion es o no apto para concretar su plan para conquistar el mundo. De esta manera ha etiquetado más de 500 minions. Lamentablemente, ha perdido dicha fórmula y necesita seguir decidiendo si nuevos minions son o no aptos para su macabro plan.

Es por esto que nuestro objetivo será construir clasificadores que estimen lo mejor posible la probabilidad de que nuevos minions sean o no aptos para concretar el plan de conquista y así facilitarle las cosas al doctor Nefario.

Por otra parte, ya que el doctor Nefario tuvo problemas con equipos que sobreestiman sus resultados, decidió guardarse varias etiquetas extra que no compartirá con nadie, y que luego utilizará para elegir al mejor equipo, al cual contratará para (de una vez por todas) conquistar el mundo. 


En concreto:

Tendrán disponible una matriz de datos $X$ de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$. Además, tendrán y, un vector de $500$ posiciones con dos posibles valores: $True$ y $False$. 

Por otra parte, tendrán disponibles más instancias de evaluación $X_{competencia}$ sin las respectivas etiquetas que utilizaremos para evaluar sus resultados. 



In [ ]:
# PREAMBULOS
%matplotlib inline

SEED = 1234

# Commons imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import Image, display, HTML
from sklearn.externals.six import StringIO
import pydotplus
from time import time
import itertools
import graphviz

from scipy.stats import randint as sp_randint, normaltest

import numpy as np
np.set_printoptions(precision=4)
np.random.seed(SEED)

import pandas as  pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
pd.set_option('precision', 4)

import matplotlib.pyplot as plt
import seaborn as sns


import sklearn.model_selection

## Classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

import sklearn.ensemble
import sklearn.svm

In [ ]:
# Common functions
def dibujar_arbol(clf):
    dot_data = StringIO()
    sklearn.tree.export_graphviz(clf, out_file=dot_data,  
                    filled=True,
                    special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    display(Image(graph.create_png()))

In [67]:
# Carga de datos
X = pd.read_csv("X.csv", index_col="index")
Y = pd.read_csv("y.csv", index_col="index", dtype=int)  # Cargamos los valores booleanos (True y False)
                                                        # como números (1 y 0) para facilitar el manejo luego. 
    
X_competencia = pd.read_csv("X_competencia.csv", index_col="index")
Y_competencia_ejemplo = pd.read_csv("y_competencia_ejemplo.csv", index_col="index")

# Descomentar si quieren ver los datos para la competencia:
# display(X_competencia) 
# display(y_competencia_ejemplo)

,0,1,2,3,4,5,6,...,193,194,195,196,197,198,199
count,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,...,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000
mean,0.0384,0.0715,0.0056,-0.0103,-0.0436,-0.0208,-0.0571,...,-0.0187,0.0087,-0.0356,-0.1940,0.0250,0.0257,-0.0036
std,1.0153,0.9613,1.0360,1.0230,1.0647,0.9898,1.0000,...,0.9731,0.9716,1.0075,1.0246,0.9934,0.9940,0.9819
min,-3.1722,-2.4596,-2.8834,-3.7474,-2.9987,-3.2014,-3.6855,...,-2.9110,-2.9642,-2.5163,-3.9278,-2.4254,-2.6234,-2.8690
25%,-0.6090,-0.6207,-0.7180,-0.6594,-0.7177,-0.6510,-0.7073,...,-0.6441,-0.6509,-0.6613,-0.8689,-0.6466,-0.6354,-0.6855
50%,0.0602,0.0560,-0.0713,0.0612,-0.0097,-0.0407,-0.0771,...,-0.0473,0.0537,-0.0749,-0.1901,0.0185,-0.0332,-0.0797
75%,0.6334,0.7670,0.7066,0.6699,0.6616,0.6508,0.6029,...,0.6071,0.6860,0.5743,0.4636,0.7041,0.6575,0.6608
max,2.9702,2.7920,2.6905,2.8091,2.9823,2.9342,3.3240,...,3.0983,3.1469,2.9109,2.4942,3.1804,3.0034,2.5107


,output
count,500.0000
mean,0.4580
std,0.4987
min,0.0000
25%,0.0000
50%,0.0000
75%,1.0000
max,1.0000


## Análisis de los datos

In [69]:
print('Descripción del DataFrame con (muestras x predictores)')
display(X.describe())

Descripción del DataFrame con (muestras x predictores)


,0,1,2,3,4,5,6,...,193,194,195,196,197,198,199
count,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,...,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000,500.0000
mean,0.0384,0.0715,0.0056,-0.0103,-0.0436,-0.0208,-0.0571,...,-0.0187,0.0087,-0.0356,-0.1940,0.0250,0.0257,-0.0036
std,1.0153,0.9613,1.0360,1.0230,1.0647,0.9898,1.0000,...,0.9731,0.9716,1.0075,1.0246,0.9934,0.9940,0.9819
min,-3.1722,-2.4596,-2.8834,-3.7474,-2.9987,-3.2014,-3.6855,...,-2.9110,-2.9642,-2.5163,-3.9278,-2.4254,-2.6234,-2.8690
25%,-0.6090,-0.6207,-0.7180,-0.6594,-0.7177,-0.6510,-0.7073,...,-0.6441,-0.6509,-0.6613,-0.8689,-0.6466,-0.6354,-0.6855
50%,0.0602,0.0560,-0.0713,0.0612,-0.0097,-0.0407,-0.0771,...,-0.0473,0.0537,-0.0749,-0.1901,0.0185,-0.0332,-0.0797
75%,0.6334,0.7670,0.7066,0.6699,0.6616,0.6508,0.6029,...,0.6071,0.6860,0.5743,0.4636,0.7041,0.6575,0.6608
max,2.9702,2.7920,2.6905,2.8091,2.9823,2.9342,3.3240,...,3.0983,3.1469,2.9109,2.4942,3.1804,3.0034,2.5107


A primera vista parece que los predictores tienen una distribución gaussiana con media $\approx 0$ y varianza $\approx 1$. Se puede realizar un test de normalidad aunque esto sólo puede permitirnos rechazar la hipótesis nula: "los datos provienen de una distribución normal con media $\mu$ y varianza $\sigma^2$".

In [83]:
def test_normality_on(values, alpha=0.05):
    """
    Computes for each features a normality test.
    
    This function tests the null hypothesis (or H0) that a sample comes
    from a normal distribution.
    
    Parameters
    ----------
    values : array_like
        Matrix of shape (samples x features) containing the values of the features for each sample.
    
    pvalue : float
        A threshold to reject or not H0.
    
    Returns
    -------
    h0_is_rejected : array-like
        A bit-mask which has each index has value 1 if H0 is rejected for the given alpha value,
        0 in other case.
    """
    # we use two different tests
    _statistic, _p_values = normaltest(values)
    # _statistic should not live in [0,4] to reject H0. _k2 is non-negative array.
    # _p_values should be lower than alpha to reject H0. _k2 is an array.
    
    # rejects if at least one of both rejects
    return np.array(
        [
            _p_values[_ith_feature] < alpha or 4 < _statistic[_ith_feature] 
             for _ith_feature in range(len(_p_values))
        ]
    )

normality_is_rejected = test_normality_on(X_train_np)
print(normality_is_rejected.sum())

19


In [ ]:
display(Y.describe())

TODO: algún análisis del input no estaria mal

posibilidades:
1) no se puede descartar que sean normales
2) correspondencia con normales con media 0 y varianza 1
3) correlación entre variables?
4) porcentaje de cada clase en la muestra


## Ejercicio 1

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. En este punto pedimos que evalúen cómo separar sus datos para desarrollo y para evaluación tomando en cuenta la competencia. 

In [ ]:
percentage_train = 9/10
X_train, X_held_out, Y_train, Y_held_out = sklearn.model_selection.train_test_split(
    X, Y, train_size=percentage_train, stratify=Y
)

# Just to check sizes and percentage of class 1 in each one
print("Tamaño conj de entrenamiento: {}, tamaño conj de clases de entrenamiento: {}. porcentaje de clase 1 en conj {:.2f}.".format(
        X_train.shape, Y_train.shape, int(Y_train.sum())/len(Y_train))
)
print("Tamaño conj de test: {}, tamaño conj de clases de test: {}, porcentaje de clase 1 en conj {:.2f}.".format(
        X_held_out.shape, Y_held_out.shape, int(Y_held_out.sum())/len(Y_held_out))
)

plt.figure(figsize=(5, 3))
plt.hist(np.array(Y_held_out))  # muestra un histograma para la distribución de y.
plt.show()

## Ejercicio 2

### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo árbol de decisión, de manera de obtener una estimación realista de la performance de los mismos. 

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default. 
2. Estimar la performance del modelo utilizando K-fold cross validation con K = 5, con las métricas “Accuracy” y “ROC AUC”. Para ello, se pide medir la performance en cada partición tanto sobre el fold de validación como sobre los folds de entrenamiento. Luego, completar la primera tabla.
3. Entrenar árboles de decisión para cada una de las siguientes combinaciones y completar la segunda tabla.

----

**EJERCICIO EXTRA: Usar la implementación de árboles de decisión que realizaron para la guía de ejercicios de la materia. Adaptarla para que cumpla con la interfaz requerida por sklearn, asegurarse de que funcione con variables continuas y reproducir las tablas anteriores.   **

----

In [ ]:
def plot_accuracy_and_auc_roc_on_train_and_test(metrics_results,
        tabble_header='Resultado Ejercicio 2.2: accuracy y roc auc en árboles de decisión con altura máxima de 3'):
    """
    Param metrics_results :type: dict should at least contains keys 
    ['train_accuracy', 'test_accuracy', 'train_roc_auc', 'test_roc_auc'].
    """
    
    df = pd.DataFrame(index=range(1,6))
    df.index.name = "Fold de validación"

    df["Accuracy (entrenamiento)"] = metrics_results['train_accuracy']
    df["Accuracy (validación)"] = metrics_results['test_accuracy']
    df["AUC ROC (entrenamiento)"] = metrics_results['train_roc_auc']
    df["AUC ROC (validación)"] = metrics_results['test_roc_auc']
    display(HTML('<h3> {} </h3>'.format(tabble_header)))
    display(df)

    fig = df.plot(kind="bar")
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
    plt.show()
    
    return

X_train_np = np.array(X_train)
Y_train_np = np.array(Y_train).ravel()


# In order to avoid using the same variable's names in different excercises,
# each excercise is a function with local variables

def excercise_2_2():
    decision_tree = DecisionTreeClassifier(max_depth=3)

    metric_scores = sklearn.model_selection.cross_validate(
        decision_tree, X_train_np, Y_train_np, scoring=('roc_auc', 'accuracy'), cv=5, n_jobs=4
    )

    plot_accuracy_and_auc_roc_on_train_and_test(metric_scores)
    return

excercise_2_2()

In [ ]:
def plot_table_excercise_2_3(metrics_scores,
                             table_header='Resultado Ejercicio 2.3: árboles de decisión con diferentes hiperparámetros'):
    """
    Param metrics_scores :type: dict should at least has 'train_results' and 'test_results' keys
    """
    df = pd.DataFrame(index=range(0,6))

    df['Altura Máxima'] = [3, 5, 'Inifinity'] * 2
    df['Medida de Separación'] = ['Gini'] * 3 + ['Ganancia de Información'] * 3
    df['Promedio AUC ROC (entrenamiento)'] = metrics_scores['train_results']
    df['Promedio AUC ROC (validación)'] = metrics_scores['test_results']
   
    display(HTML("<h3> {} </h3>".format(table_header)))
    display(df)
    
    df.plot(kind="bar")
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
    plt.show()
    
    return

def excercise_2_3():
    metric_scores = { 'train_results' : [], 'test_results' : [] }
            
    for _criterio, _depth in itertools.product(['gini', 'entropy'], [3, 5, None]):
        
        decision_tree = DecisionTreeClassifier(max_depth=_depth, criterion=_criterio)
        
        tree_metric_score = sklearn.model_selection.cross_validate(
            decision_tree, X_train_np, Y_train_np, scoring='roc_auc', cv=5
        )
            
        metric_scores['train_results'].append(np.mean(tree_metric_score['train_score']))
        metric_scores['test_results'].append(np.mean(tree_metric_score['test_score']))
    
    plot_table_excercise_2_3(metric_scores)
        
excercise_2_3()

## Ejercicio 3: Comparación de algoritmos


Se pide explorar distintas combinaciones de algoritmos de aprendizaje e hiperparámetros, de manera de buscar una performance óptima. Para este ejercicio es necesario que evalúen posibilidades utilizando la técnica de Grid Search. Como métrica de performance, usar siempre el área bajo la curva (AUC ROC) resultante de 5-fold cross-validation. 

Algoritmos a probar: KNN, árboles de decisión, LDA, Naive Bayes y SVM. Hiperparámetros: Revisar la documentación de cada uno para la búsqueda de combinaciones prometedoras.  

Se pide generar un reporte que contenga: 

1. Una descripción de las distintas combinaciones consideradas y su performance asociada (las que consideren relevantes, con al menos la mejor combinación para cada algoritmo). 

1. Una breve explicación de los factores que creen que produjeron dicho resultado. 

En este punto evaluaremos tanto los hiperparámetros elegidos como las conclusiones relacionadas a por qué piensan que ciertos algoritmos funcionan mejor que otros para estos datos. 



----

**EJERCICIO EXTRA**: Utilizar RandomizedSearchCV con rangos de parámetros que contengan a los utilizados en el GridSearch. Analizar si se encontraron mejores combinaciones de parámetros que no hayan sido tenidas en cuenta con el GridSearch y cuál fue la diferencia de tiempo de ejecución. 

----

In [ ]:
def top_results(grid, top=5, skip=0):
    print("Top {} Salteando {} combinaciones".format(top, skip))
    df = pd.DataFrame(grid.cv_results_["params"])
    df["mean_score_validation"] = grid.cv_results_["mean_test_score"]
    df["mean_score_training"] = grid.cv_results_["mean_train_score"]
    display(df.sort_values(by="mean_score_validation", ascending=False).head(top+skip).tail(top))
    return

def apply_grid_search_on_classifier_and_parameters(
    classifier, classifier_parameters, _X_train, _Y_train, metrics_names=('roc_auc'), cv=5, 
    n_jobs=4, error_score='raise'):
    
    _clf = GridSearchCV(
        classifier, classifier_parameters, scoring=metrics_names, cv=cv, n_jobs=n_jobs,
        error_score=error_score
    )
    
    return _clf.fit(_X_train, _Y_train)    

### Grid Search sobre clasificadores

In [ ]:
# A function to apply grid on each Classifier with its own possible arguments  

def knn_grid_search_parametrization(_X_train, _Y_train):
    knn_arguments = [
        {
            'algorithm': ['brute'],
            'n_neighbors': range(1, 350, 50),
            'weights': ['distance','uniform']
        },
        {
            'algorithm': ['ball_tree', 'kd_tree'],
            'n_neighbors': range(1, 350, 50),
            'weights': ['distance','uniform'],
            'p': [1, 2, 200],
            'leaf_size': range(30, 150, 30)
        }
    ]
    
    knn_classifier = KNeighborsClassifier()
    
    return apply_grid_search_on_classifier_and_parameters(knn_classifier, knn_arguments, _X_train, _Y_train)

### Resultados

### Árboles de decisión

In [ ]:
def tree_grid_search_parametrization(_X_train, _Y_train, _error_score='raise'):
    
    _amount_of_features_in_dataset = _X_train.shape[1]
    _possible_max_features_ = list(range(25, min(200, _amount_of_features_in_dataset), 10)) \
        + list(filter(lambda x: x is None or x <= _amount_of_features_in_dataset, [7, 14, 20, None]))
    tree_arguments = [
        {
            'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_depth': list(range(3, 13, 3)) + list(range(15, 36, 10)) + [None],
            'max_features': _possible_max_features_,
            'max_leaf_nodes': list(range(10, 501, 50)) + [None]
        }
    ]
    
    tree_classifier = DecisionTreeClassifier(random_state = SEED) # it doesn't seem to be taking numpy.seed
    
    return apply_grid_search_on_classifier_and_parameters(
        tree_classifier, tree_arguments, _X_train, _Y_train, error_score=_error_score
    )

#grid_tree = tree_grid_search_parametrization(X_train_np, Y_train_np)
top_results(grid_tree,10)

### Parámetros
Se varía la forma de medir la calidad de una separación (criterion) en un nodo como también la forma en la que se elige entre las posibles (splitter). Se varía también la cantidad máxima de predictores a considerar para buscar la mejor separación. 
Nos pareció de interés variar la altura y la cantidad de hojas permitidas en el árbol para ver como generaliza una árbol pequeño o con pocas hojas respecto a uno sin limitación.

### Resultados
Se obtuvieron mejores resultados en los datos de validación respecto a los obtenidos en el ejercicio 2.2 y 2.3 con resultados en los datos de entrenamiento menores (el top 10 está por debajo de %90).

Si nuestros datos hubieran sido linealmente separables, hubieramos obtenido mejores resultados.
    
    
Los predictores son permutados en cada nodo al momento de buscar una separación, por lo que considerar diferentes de no fijar una semilla, mismos parámetros y datos pueden concluir en árboles distintos. Se fijo el parámetro $\textit{random_state}$ con el mismo valor que la seed de numpy para obtener de manera determinista un árbol.

Algunas preguntas que podemos hacernos son

1. ¿Qué features separa y cómo el mejor árbol obtenido?
2. ¿Cómo afecta el cambiar de algunos hiperaparámetros?
    + ¿Cómo cambia el auc roc mientras aumentamos la profundidad máxima del árbol?
    + ¿Cómo cambia el auc roc mientras aumentamos la cantidad de hojas máxima del árbol?

1)¿Qué features separa y cómo el mejor árbol obtenido?

In [ ]:
## Functions for DecisionTreeClassifier and its internal Tree struct

def create_decision_tree_classifier_with_params(parameter_combination):
    criterion = parameter_combination['criterion']
    max_depth = parameter_combination['max_depth']
    max_features = parameter_combination['max_features']
    max_leaf_nodes = parameter_combination['max_leaf_nodes']
    splitter = parameter_combination['splitter']
    
    return DecisionTreeClassifier(
            criterion=parameter_combination['criterion'],
            max_depth= None if max_depth is None or np.isnan(max_depth) else int(max_depth),
            max_features= None if max_features is None or np.isnan(max_features) else int(max_features),
            max_leaf_nodes= None if max_leaf_nodes is None or np.isnan(max_leaf_nodes) else int(max_leaf_nodes),
            splitter=parameter_combination['splitter'],
            random_state=SEED
    )

def calculate_nodes_depth_and_leaves(tree_classifier):
    """
    Param tree_classifier :type: DecisionTreeClassifier.
    
    Traverse the internal tree_ structure tree_classifier in order to compute, its depth, amount of nodes
    and amount of leaves.
    
    Trivial tree (only one node) has depth equals to 0.
    """
    n_nodes = tree_classifier.tree_.node_count
    children_left = tree_classifier.tree_.children_left
    children_right = tree_classifier.tree_.children_right

    # The tree structure can be traversed to compute various properties such
    # as the depth of each node and whether or not it is a leaf.
    max_depth_used = -1
    leaves_amount = 0
    stack = [(0, max_depth_used)]  # (root_node_id, its_parent_depth)
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        
        max_depth_used = max(max_depth_used, parent_depth+1) # keep max depth
        
        # If it's a leaf its childs are equal
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            leaves_amount += 1
    
    return n_nodes, max_depth_used, leaves_amount # nodes_count, max_depth_used, leaves_count 

In [ ]:
def print_first(grid, top=10):
    df = pd.DataFrame(grid.cv_results_["params"])
    df["mean_score_validation"] = grid.cv_results_["mean_test_score"]
    df["mean_score_training"] = grid.cv_results_["mean_train_score"]
    df = df.sort_values(by="mean_score_validation", ascending=False).head(top)
    
    for _, _parameters in df.iterrows():
        print(_parameters)
        best_tree = create_decision_tree_classifier_with_params(_parameters)
        best_tree.fit(X_train_np, Y_train_np)
        print('Cantidad de nodos {}\nAltura del árbol {}\nCantidad de hojas {}'.format(
                *calculate_nodes_depth_and_leaves(best_tree))
        )
        dibujar_arbol(best_tree)
    return

# plot first 10 trees to see their shape and the features they used to split in each node 
print_first(grid_tree,1)

### Observación:
En el top 10 vemos presentes muchos arboles con criterio $\textit{entropy}$ con el mismo valor de AUC ROC. Esto es sólo ocurre porque encontramos el mismo árbol con una altura final de 3 repetido para cada $\textit{max_depth}$. No se expande más debido al threshold de mínima reducción de impuereza (default = 0.0).

Si evitamos estos resultados con misma altura, podremos ver

2) ¿Cómo afecta el cambiar de algunos hiperaparámetros?
    - ¿Cómo cambia el auc roc mientras aumentamos la profundidad máxima del árbol?
    - ¿Cómo cambia el auc roc mientras aumentamos la cantidad de hojas máxima del árbol?
    

Para ver la primera pregunta consideraramos los árboles sin límite en la cantidad de hojas y en la segunda los árboles sin límite en la cantidad profundidad. Creamos un árbol para cada combinación de hiperparámetros utilizando la misma semilla pero en este caso fiteamos sobre todo el set de datos de entrenamiento y no sobre un fold sabiendo que el árbol no será exactamente el mismo pero 'similar' (el valor que nos muestra GridSearch es un promedio de los obtenidos por los folds).

In [ ]:
def add_nodes_depth_and_leaves_features_of_tree_for_each_parameter_combination(
    parameter_combination_df, _X_train, _Y_train):
    """
    Param parameter_combination_df :type: DataFrame contains a serie of hiperparameters to create
    DecisionTreeClassifiers.
    
    For each one compute the depth and amount of leaves and nodes of the internval tree structure.
    This is information is returned in the input DataFrame as new columns.
    """
    
    amount_nodes = np.zeros(shape=len(parameter_combination_df.index), dtype=np.int64)
    depth_used = np.zeros(shape=len(parameter_combination_df.index), dtype=np.int64)
    amount_leaves = np.zeros(shape=len(parameter_combination_df.index), dtype=np.int64)
    
    _index=0
    for _, parameter_combination in parameter_combination_df.iterrows():
        _tree_classifier = create_decision_tree_classifier_with_params(parameter_combination)
        _tree_classifier.fit(X_train, Y_train)
        amount_nodes[_index], depth_used[_index], amount_leaves[_index] = calculate_nodes_depth_and_leaves(_tree_classifier)
        _index +=1
    
    parameter_combination_df['amount_nodes'] = amount_nodes
    parameter_combination_df['depth_used'] = depth_used
    parameter_combination_df['amount_leaves'] = amount_leaves
    
    return parameter_combination_df

def plot_dataframe_as_bar(dataframe):
    fig = dataframe.plot(kind="bar", figsize=(18.5, 10.5))
    plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
    plt.show()
    return

def plot_score_while_increasing_depth(df_without_depth_bound, _X_train, _Y_train):
    df_without_depth_bound = add_nodes_depth_and_leaves_features_of_tree_for_each_parameter_combination(
        df_without_depth_bound, _X_train, _Y_train
    )
    
    df_without_depth_bound = df_without_depth_bound.drop(
        columns=['max_depth']
    )[['depth_used','mean_score_validation', 'mean_score_training']].groupby(['depth_used']).mean()
    
    plot_dataframe_as_bar(df_without_depth_bound)
    return

def plot_score_while_increasing_leaves(df_without_leaves_bound, _X_train, _Y_train):
    df_without_leaves_bound = add_nodes_depth_and_leaves_features_of_tree_for_each_parameter_combination(
        df_without_leaves_bound, _X_train, _Y_train
    )
    
    df_without_leaves_bound = df_without_leaves_bound.drop(
        columns=['max_leaf_nodes']
    )[['amount_leaves','mean_score_validation', 'mean_score_training']].groupby(['amount_leaves']).mean()
    
    plot_dataframe_as_bar(df_without_leaves_bound)
    return
    
def plot_score_while_increasing_depth_and_max_feature(grid, top=2):
    df = pd.DataFrame(grid.cv_results_["params"])
    df["mean_score_validation"] = grid.cv_results_["mean_test_score"]
    df["mean_score_training"] = grid.cv_results_["mean_train_score"]
    
    df_without_depth_bound = df[np.isnan(df['max_depth'])].head(top)
    plot_score_while_increasing_depth(df_without_depth_bound, X_train_np, Y_train_np)
    
    df_without_leaves_bound = df[np.isnan(df['max_leaf_nodes'])].head(top)
    plot_score_while_increasing_leaves(df_without_leaves_bound, X_train_np, Y_train_np)
    return

plot_score_while_increasing_depth_and_max_feature(grid_tree, len(grid_tree.cv_results_["mean_test_score"]))

Se puede ver que aumentar el límite de hojas y altura del árbol aumenta en promedio el valor de la métrica auc roc a %100 (aproximadamente) sobre los datos de entrenamiento de manera rápida (cantidad de hojas mayor a 30 o profundidad mayor a 8) mientras que la misma métrica sobre los datos de validación empeoran.

Podemos justificar esto en el overfitting de los árboles al no limitar el tamaño y de los mismos. "Árboles más pequeños, generalizan mejor".

### Naive-Bayes

In [ ]:
def naive_bayes_grid_search_parametrization(_X_train, _Y_train):
    """
    The only parameter of Naive-Bayes classifier are the prior probabilities of each class,
    which are automatically calculated as #items_class_1 / #items and #items_class_2 / #items
    if no parameter is specified.
    
    I doesn't make sense trying other prior probabilities than the real ones if the data isn't changing.
    """
    naive_bayes_classifier = GaussianNB()
    naive_arguments = [{}] #None
    
    return apply_grid_search_on_classifier_and_parameters(
        naive_bayes_classifier, naive_arguments, _X_train, _Y_train
    )

#grid_naive = naive_bayes_grid_search_parametrization(X_train_np, Y_train_np, metric_name='roc_auc')
top_results(grid_naive)

### Parámetros

Los parámetros de este clasificador son las probabilidades a priori de cada clase. Esta es información que no conocemos pero que podríamos estimar utilizando la frecuencia relativa de nuestra muestra de entrenamiento. Esto mismo es lo que hace el clasificador si ningún parámetro es especificado.

### Resultados

Los resultados son mejores que los obtenidos utilizando árboles de decisión.

Este método tiene dos asunciones:
    1. Los predictores son independientes entre sí (propio del método)
    2. Las probabilidad condicional P(X_i=x_i/Y=k) de observar el valor x_i para el predictor X_i dado que la muestra x pertenece a la clase k se comporta como una gaussiana (propio de la forma en que está implementado en python)

1) Para responder la primera asunción podemos calcular la matriz de covarianza de los predictores con los datos entrenamiento. Si dos variables independientes su covarianza es 0. No vale la vuelta pero podemos usar el contrarrecíproco: si la covarianza entre dos predictores es distinta 0 entonces no son independientes.

In [ ]:
def plot_correlation_between_dataframe_columns(df_features):
    corr = df_features.corr()
    fig = plt.figure(figsize=(10, 10), dpi=200)
    ax = fig.add_subplot(111)
    cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
    fig.colorbar(cax)
    ticks = list(map(int, np.linspace(0,len(X_train.columns)-1,20)))
    ax.set_xticks(ticks)
    plt.xticks(rotation=90)
    ax.set_yticks(ticks)
    ax.set_xticklabels(df_features.columns[ticks])
    ax.set_yticklabels(df_features.columns[ticks])
    plt.show()
    return

plot_correlation_between_dataframe_columns(X_train)

El gráfico anterior nos muestra que hay correlación lineal mayor a 0.5 entre varios predictores, y con esto no independencia entre los mismos. No podemos decir nada respecto a aquellos predictores $P_1$, $P_2$, con $|\text{Covarianza}(P_1,P_2)| < 0 + \textit{epsilon}$

donde $\textit{epsilon}$ es algún valor que admita un error (ej: [0, 0.05])

2) Para observar si era un error asumir que podían aproximarse las probabilidades condicionales con gaussianas se puede 

In [ ]:
def plot_naive_bayes_information(_X_train, _Y_train):
    naive_bayes_classifier = GaussianNB().fit(_X_train, _Y_train)
    
    for i in naive_bayes_classifier.classes_:
        print('P(Y={}) = {:.2f}'.format(i, naive_bayes_classifier.class_prior_[i]))
    
    _df_naive = pd.DataFrame(
            data=
            {
                'clase_0_media': naive_bayes_classifier.theta_[0],
                'clase_1_media': naive_bayes_classifier.theta_[1],
                'clase_0_varianza': naive_bayes_classifier.sigma_[0],
                'clase_1_varianza': naive_bayes_classifier.sigma_[1],
            }
    )
    print(_df_naive.describe())
    return _df_naive

def test_normality_on(values, alpha=0.05):
    _k2, _p = normaltest(values)
    return np.array([1 if _p[_index] < alpha or 4 < _k2[_index] else 0 for _index in range(len(_p))])

#plot_naive_bayes_information(X_train, Y_train)

# see the features when the known label is a fixed class
for _clase in np.unique(Y_train_np):
    print('Descripción de los datos de entrenamiento cuando la clase es {}'.format(_clase))
    print(X_train[Y_train_np==_clase].describe())

alpha_value = 0.05
_seems_not_normal_0 = test_normality_on(X_train_np[Y_train_np==0], alpha_value)
_seems_not_normal_1 = test_normality_on(X_train_np[Y_train_np==1], alpha_value)

print('Se puede rechazar que {} probabilidades condicionales P(X_i=x_i /Y=0) provengan de una \
distribución normal con un nivel de significancia de {}'.format(_seems_not_normal_1.sum(), alpha_value))

print('Se puede rechazar que {} probabilidades condicionales P(X_i=x_i /Y=1) provengan de una \
distribución normal con un nivel de significancia de {}'.format(_seems_not_normal_0.sum(), alpha_value))

print('De estos casos se intersecan {} predictores'.format(_seems_not_normal_0[_seems_not_normal_1].sum()))

Esto nos permite decir que no valían todas las asunciones hechas al utilizar el método. Sin embargo el valor promedio AUC ROC obtenido sobre los datos de validación fue mejor que al utilizar árboles de decisión.

### KNN

In [ ]:
#grid_knn = knn_grid_search_parametrization(X_train_np, Y_train_np)
top_results(grid_knn, 10)

### Linear Discriminant Analysis

In [56]:
# GridSearch
def linear_discriminant_analysis_grid_search_parametrization(_X_train, _Y_train):
    lda_arguments = [
        {'solver': ['svd']},
        {
            'solver': ['lsqr', 'eigen'],
            'shrinkage': list(np.linspace(0,1,11)) + [None,'auto']
        }
    ]
    
    lda_classifier = LinearDiscriminantAnalysis()
    
    return apply_grid_search_on_classifier_and_parameters(lda_classifier, lda_arguments, _X_train, _Y_train)
    
#grid_lda = linear_discriminant_analysis_grid_search_parametrization(X_train_np, Y_train_np)
top_results(grid_lda, 10)

Top 10 Salteando 0 combinaciones


,shrinkage,solver,mean_score_validation,mean_score_training
15,0.7,lsqr,0.8452,0.9445
16,0.7,eigen,0.8451,0.9445
14,0.6,eigen,0.8431,0.9583
13,0.6,lsqr,0.8429,0.9584
17,0.8,lsqr,0.8415,0.9250
18,0.8,eigen,0.8412,0.9254
12,0.5,eigen,0.8369,0.9688
19,0.9,lsqr,0.8368,0.8939
11,0.5,lsqr,0.8367,0.9688
20,0.9,eigen,0.8364,0.8945


### Parámetros
Se eligió variar el solver utilizado y utilizar valores de shrinkage cuando el solver no es SVD. Los demás parámetros se ignoran dado que no modifican la performance del clasificador.

### Resultados
Este método hace una asunción:
    + La probabilidad condicional de observar un nuevo dato x dado que pertenece a la clase k, es decir P(X=x / Y=k) tiene una distribución gaussiana con una misma matriz de covarianza.


Los resultados obtenidos en sus mejores combinaciones lo posicionan mejor que $\textbf{KNN}$ mostrándolo como una alternativa.


Es de interés probar que sucedería si intentamos reducir la dimensión de los datos utilizando LDA y luego volver a experimentar con los mismo hiperparámetros.

In [ ]:
def are_different_between_cov_matrix(_X_train, _Y_train, alpha=None):
    """
    Assume X_train_np[Y_train_np==1] = alpha * X_train_np[Y_train_np==0]
    with alpha a real number
    
    Compute alpha with two matrix norms or used it to compare
    """
    cov_1 = np.cov(X_train_np[Y_train_np==1])
    cov_0 = np.cov(X_train_np[Y_train_np==0])
    
    if alpha is None:
        alpha = np.linalg.norm(cov_1) / np.linalg.norm(cov_0)
    
    return True if alpha == np.linalg.norm(cov_1,1) / np.linalg.norm(cov_0,1) else False

if not are_different_between_cov_matrix(X_train_np, Y_train_np):
    print('Las probabilidades condicionales no comparten la misma matriz de covarianza.')

In [ ]:
def contruct_lda_classifier_from_parameters(lda_paramaters, _X_train, _Y_train, n_components=None):
    solver = lda_paramaters['solver']
    shrinkage = lda_paramaters['shrinkage']
    
    return LinearDiscriminantAnalysis(
            solver=solver,
            shrinkage= None if shrinkage is None or np.isnan(shrinkage) else float(shrinkage),
            n_components=n_components
    ).fit(_X_train, _Y_train)

def build_data_frame_from_grid_with_top(grid_lda, top=10, skip=0):    
    lda_df = pd.DataFrame(grid_lda.cv_results_['params'])
    lda_df["mean_score_validation"] = grid_lda.cv_results_["mean_test_score"]
    lda_df["mean_score_training"] = grid_lda.cv_results_["mean_train_score"]
    return lda_df.sort_values(by="mean_score_validation", ascending=False).head(top+skip).tail(top)

def get_transformed_input(_grid_lda, _X_train, _Y_train):
    _lda_X_transformed = None
    for index, parameter_comnbination in build_data_frame_from_grid_with_top(_grid_lda, 20).iterrows():
        if parameter_comnbination['solver'] == 'lsqr':
            continue
        _lda_classifier = contruct_lda_classifier_from_parameters(
            parameter_comnbination, _X_train, _Y_train, n_components=1
        )
        _lda_classifier.fit(_X_train, _Y_train)
        _lda_X_transformed = _lda_classifier.transform(_X_train)
        break
    
    print('Número original de predictores:', _X_train.shape)
    print('Número reducido de predictores:', _lda_X_transformed.shape)

    return _lda_X_transformed

def transform_X_train_and_re_run_grid(grid_lda, _X_train, _Y_train):
    _X_train_lda = get_transformed_input(grid_lda, X_train_np, Y_train_np)
    return _X_train_lda, linear_discriminant_analysis_grid_search_parametrization(_X_train_lda, _Y_train)


X_train_transformed, _grid_lda_with_transformed = transform_X_train_and_re_run_grid(
    grid_lda, X_train_np, Y_train_np
)

print('\nCorriendo LDA con los datos de entrenamiento transformados\n')

top_results(_grid_lda_with_transformed, 10)

Podemos ver que haber transformado datos de entrada, mejora considerablemente ambos scores. Lo cual también puede significar overfitting sobre los datos. Se decidió utilizar estos datos transformados con dimensión reducida como entrenamiento de otro algoritmo.

In [91]:
#print(X_train.shape)
X_train_transformed.reshape(1,450)
#print(X_train_transformed.reshape(1,450))

_df_transformed = pd.DataFrame(index=range(X_train_transformed.shape[0]), data={'col1':X_train_transformed.reshape})
#print(_df_transformed)

pd.plotting.scatter_matrix(_df_transformed, c=Y_train, s=80, figsize=(15, 8), marker='o', alpha=.8);
plt.show()

ValueError: num must be 1 <= num <= 0, not 1

In [71]:
grid_knn_transformed = support_vector_machine_grid_serch_parametrization(X_train_transformed, Y_train_np)
top_results(grid_knn_transformed)

grid_tree_transformed = tree_grid_search_parametrization(X_train_transformed, Y_train_np)
top_results(grid_tree_transformed)

grid_naive_transformed = naive_bayes_grid_search_parametrization(X_train_transformed, Y_train_np)
top_results(grid_naive_transformed)

Top 5 Salteando 0 combinaciones


,C,dual,fit_intercept,loss,penalty,mean_score_validation,mean_score_training
0,1.0000e-03,False,False,squared_hinge,l1,0.9343,0.9348
23,1.0000e+02,False,True,squared_hinge,l2,0.9343,0.9348
25,5.0000e-05,False,False,squared_hinge,l2,0.9343,0.9348
27,5.0000e-05,False,True,squared_hinge,l2,0.9343,0.9348
29,5.0000e-04,False,False,squared_hinge,l2,0.9343,0.9348


Top 5 Salteando 0 combinaciones


,criterion,max_depth,max_features,max_leaf_nodes,splitter,mean_score_validation,mean_score_training
8,gini,3.0,None,210.0,best,0.9166,0.9536
6,gini,3.0,None,160.0,best,0.9166,0.9536
14,gini,3.0,None,360.0,best,0.9166,0.9536
18,gini,3.0,None,460.0,best,0.9166,0.9536
12,gini,3.0,None,310.0,best,0.9166,0.9536


Top 5 Salteando 0 combinaciones


,mean_score_validation,mean_score_training
0,0.9343,0.9348


### Linear Support Vector Machine

In [63]:
# GridSearch
def support_vector_machine_grid_serch_parametrization(_X_train, _Y_train):
    C_range = [10**e for e in range(-3, 3)] + [0.05 * 10**e for e in range(-3, 3)]
    SVM_Linear_arguments = [
        {
            'C': C_range,
            'loss': ['squared_hinge'],
            'penalty': ['l1', 'l2'],
            'dual': [False],
            'fit_intercept': [False, True]
        }
    ]

    linearsvm_classifier = sklearn.svm.LinearSVC()

    return apply_grid_search_on_classifier_and_parameters(
        linearsvm_classifier, SVM_Linear_arguments, _X_train, _Y_train
    )

grid_svm_linear = support_vector_machine_grid_serch_parametrization(X_train_np, Y_train_np)
top_results(grid_svm_linear)

Top 5 Salteando 0 combinaciones


,C,dual,fit_intercept,loss,penalty,mean_score_validation,mean_score_training
36,0.0500,False,False,squared_hinge,l1,0.8502,0.9687
38,0.0500,False,True,squared_hinge,l1,0.8494,0.9688
1,0.0010,False,False,squared_hinge,l2,0.8444,0.9568
3,0.0010,False,True,squared_hinge,l2,0.8444,0.9572
31,0.0005,False,True,squared_hinge,l2,0.8438,0.9354


### Parámetros:

Se eligió ese rango específico de valores de C para permitirle a SVM 'ignorar' muestras de entrenamiento cerca del margen y permitirse colocarlas en el lado equivocado (lo que ocurre con valores C chicos). Esto sirve especialmente en los casos donde hay mucho 'ruido' presente en la muestra.

Por otro lado se recomienda utilizar valores de C grandes cuando la penalidad es $\textit{l1}$ para que más predictores sean considerados (http://scikit-learn.org/stable/modules/svm.html#tips-on-practical-use)

### Resultados:

El utilizar un C chico, le permitió superar a los clasificadores previos. Esto nos hace dudar de la existencia de ruido en los datos de entrenamiento (sumado a lo ya planteado de no ser linealmente separable).


### Observaciones:
SVM necesita que los datos esten en la misma escala. Para la corrida, ig



In [ ]:
# Generic SVM
def generic_support_vector_machine_grid_search_parametrization(_X_train, _Y_train):
    C_range = [0.01 * 10**e for e in range(6)] + [0.05 * 10**e for e in range(6)]
    SVM_General_arguments = [
        {
            'kernel': ['rbf'],
            'C': C_range,
            'gamma': ['auto'],#list(np.linspace(0.0001, 1, 10)),
            'shrinking': [True, False]
        },
        {
            'kernel': ['sigmoid'],
            'coef0': np.linspace(0,10,11),
            'C': C_range,
            'gamma': ['auto'],#list(np.linspace(0.0001, 1, 10)),
            'shrinking': [True, False]
        },
        {
            'kernel': ['poly'],
            'coef0': np.linspace(0,10,11),
            'gamma': ['auto'],#list(np.linspace(0.0001, 1, 10)),
            'degree': np.linspace(2,10,9),
            'C': C_range,
            'shrinking': [True, False]
        }
    ]
    """
        {
            'kernel': ['precomputed'],
            'C': np.linspace(0.01,1000,11),
            'shrinking': [True, False]
        },
    """

    general_svm_classifier = sklearn.svm.SVC()
    
    return apply_grid_search_on_classifier_and_parameters(
        general_svm_classifier, SVM_General_arguments, _X_train, _Y_train
    )

grid_svm_general = generic_support_vector_machine_grid_search_parametrization(X_train_np, Y_train_np)
top_results(grid_svm_general, 10)

## Conclusiones Grid Search



In [ ]:
#### Conclusiones Grid Search

def print_top(dicc, n):
    values = [(v,k,) for k,v in dicc.items()]
    print(sorted(values, key=lambda x: 1/x[0])[:10])
    
print_top(resultados_training, 10)
print_top(resultados_validation, 10)

#< < COMPLETAR > >

## Ejercicio 4: 
### Diagnóstico Sesgo-Varianza. 

En este punto, se pide inspeccionar dos de sus mejores modelos encontrados hasta ahora: el mejor modelo de tipo árbol de decisión y el mejor de tipo SVM. Para ello:

1. Graficar curvas de complejidad para cada modelo, variando la profundidad en el caso de árboles, y el hiperparámetro C en el caso de SVM. Diagnosticar cómo afectan al sesgo y a la varianza esos dos hiperparámetros.
2. Graficar curvas de aprendizaje para cada modelo. En base a estas curvas, sacar conclusiones sobre si los algoritmos parecen haber alcanzado su límite, o bien si aumentar la cantidad de datos debería ayudar.
3. Construir un modelo RandomForest con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos (usar  grid search para encontrar una buena combinación de parámetros).  


**Atención**: Tener en cuenta que debemos seguir utilizando ROC AUC como métrica para estas curvas.

**ver**: http://scikit-learn.org/stable/modules/learning_curve.html#learning-curve

----
**EJERCICIO EXTRA:** Utilizar RandomizedSearchCV para explorar la performance del algoritmo de Gradient Boosting y comparar con los resultados obtenidos en el punto (c).


----




## Competencias

La entrega del trabajo estará acompañada de una competencia en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

Su tarea será estimar la performance (AUC ROC) que tendrá su mejor modelo en datos de evaluación (X_competencia). 

Para ello, deberán predecir las probabilidades de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUC ROC resultante y calcularemos el resultado real. El grupo que consiga acercarse más al valor real, será el grupo ganador.  

Recomendamos no perder de vista esta competencia en el momento de separar los datos en los primeros puntos. 

Para esto, junto con la entrega del informe, deberán enviar un archivo en formato csv con las columnas “index” y “output” (ver ejemplo de archivo en: [y_competencia_ejemplo.csv](https://github.com/pbrusco/aa-notebooks/blob/master/TP1/y_competencia_ejemplo.csv)) y un valor esperado de AUC ROC. 


## Entrega
- Contarán con un esqueleto en formato Jupyter Notebook en donde tendrán que completar las celdas faltantes (ya sea con explicaciones y gráficos o código). 
- El notebook final deberá ser entregado en formatos .html e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (Kernel - Restart and Run All) utilizando las bibliotecas requeridas en el archivo: requirements.txt del repositorio. 
- Tienen tiempo hasta las 23:59hs del día miércoles 17/10/2018. La entrega se debe realizar a través del campus virtual y debe contener el informe.
- El trabajo deberá elaborarse en grupos de 3 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.
- En el primer parcial se incluirá una pregunta sobre la solución entregada. Esa pregunta no influirá en la nota del parcial, pero sí en la nota individual del TP1.
- La participación en la competencia es obligatoria. De todas maneras, el resultado no incidirán en la nota de la materia.
- Los ejercicios extra son opcionales para aprobar el TP, pero son obligatorios para promocionar la materia.
